# Demo: ip衍生

In [6]:
# IPSeeker 单机测试

from IPSeeker import DeriveIp

# path = 'ipseeker.dat'
path = 'qqwry.dat'

deriveip = DeriveIp(path)

ip = '14.215.177.39'
city, carrier = deriveip.derive(ip)

print city
print carrier

广州
北京百度网讯科技有限公司电信节点


In [9]:
# IPSeeker  spark测试

from IPSeeker import DeriveIp

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setMaster("local[*]").setAppName("demo")
conf.set("spark.driver.memory", "5g")
conf.set('spark.files', './IPSeeker.py,./ipseeker.dat,./qqwry.dat') 

sqlContext = SparkSession.builder.config(conf=conf).getOrCreate()
sc = sqlContext.sparkContext

In [11]:
ip = '14.215.177.39'

rdd = sc.parallelize([ip])
rdd.take(1)

def derive_ip(x):
    deriveip = DeriveIp(path)
    city, carrier = deriveip.derive(x)
    return city, carrier

ret = rdd.map(derive_ip).collect()[0]

print ret[0]
print ret[1]

广州
北京百度网讯科技有限公司电信节点


In [3]:
# IPSeekerMaxMind 单机测试

from IPSeekerMaxMind import IPSeekerMaxMind

path = 'GeoLite2-City.mmdb'
ipseeker = IPSeekerMaxMind(path)

ip = '14.215.177.39'
country, city, longitude, latitude = ipseeker.get_common_info(ip)
print country, city, longitude, latitude

China Shenzhen 114.1333 22.5333


In [4]:
# IPSeekerMaxMind spark测试

from IPSeekerMaxMind import IPSeekerMaxMind

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setMaster("local[*]").setAppName("demo")
conf.set("spark.driver.memory", "5g")
conf.set('spark.files', './GeoLite2-City.mmdb,./IPSeekerMaxMind.py') 

sqlContext = SparkSession.builder.config(conf=conf).getOrCreate()
sc = sqlContext.sparkContext

In [5]:
ip = '14.215.177.39'

rdd = sc.parallelize([ip])
rdd.take(1)

def derive_ip(x):
    ipseeker = IPSeekerMaxMind(path)
    country, city, longitude, latitude = ipseeker.get_common_info(ip)
    return country, city

ret = rdd.map(derive_ip).collect()[0]

print ret[0]
print ret[1]

China
Shenzhen


In [ ]:
# 单例方式  （暂时还有bug）

In [1]:
from IPSeeker import DeriveIp

class DeriveIpInstance(object):
    """ 
    单例模式，为了使spark能够序列化
    """
    @classmethod
    def get_instance(cls, database_path):
        if not hasattr(cls, '_instance'):
            cls._instance = DeriveIp(database_path)
        return cls._instance

In [5]:
path = 'qqwry.dat'
deriveip = DeriveIpInstance.get_instance(path)

ip = '14.215.177.39'
city, carrier = deriveip.derive(ip)
print city, carrier

In [7]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setMaster("local[*]").setAppName("demo")
conf.set("spark.driver.memory", "5g")
# conf.set('spark.files', './IPSeeker.py,./ipseeker.dat,./qqwry.dat') 

sqlContext = SparkSession.builder.config(conf=conf).getOrCreate()
sc = sqlContext.sparkContext

In [8]:
ip = '14.215.177.39'

rdd = sc.parallelize([ip])
rdd.take(1)

def derive_ip(x):
    city, carrier = deriveip.derive(x)
    return city, carrier

ret = rdd.map(derive_ip).collect()[0]

print ret[0]
print ret[1]

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 28 in stage 4.0 failed 1 times, most recent failure: Lost task 28.0 in stage 4.0 (TID 76, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark2.2.0/python/lib/pyspark.zip/pyspark/worker.py", line 166, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/opt/spark2.2.0/python/lib/pyspark.zip/pyspark/worker.py", line 55, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark2.2.0/python/lib/pyspark.zip/pyspark/serializers.py", line 169, in _read_with_length
    return self.loads(obj)
  File "/opt/spark2.2.0/python/lib/pyspark.zip/pyspark/serializers.py", line 454, in loads
    return pickle.loads(obj)
TypeError: Required argument 'fileno' (pos 1) not found

	at org.apa